This notebook scrapes the metadata given a piece url.

In [1]:
import requests
import numpy as np
import os
from bs4 import BeautifulSoup
import re
import pandas as pd
import zipfile
import pickle
import subprocess

In [ ]:
# get the list of ids from 9_way_dataset.zip

with zipfile.ZipFile('9_way_dataset.zip', 'r') as zip_file:
    # Assuming there's only one file inside the ZIP
    filename = zip_file.namelist()[0]
    # Open the file inside the ZIP
    with zip_file.open(filename) as f:
        # Load the pickled object
        data_9 = pickle.load(f)

In [2]:
# get the list of ids from 100_way_dataset.zip

with zipfile.ZipFile('100_way_dataset.zip', 'r') as zip_file:
    # Assuming there's only one file inside the ZIP
    filename = zip_file.namelist()[0]
    # Open the file inside the ZIP
    with zip_file.open(filename) as f:
        # Load the pickled object
        data_100 = pickle.load(f)

In [ ]:
# get the list of ids from 9_way_dataset
ids_9 = []
for i in range(6,9):
    for data in data_9[i]:
        ids_9.append(data[0])

ids_9 = list(set(ids_9))
print(len(ids_9))

In [3]:
# get the list of ids from 100_way_dataset
ids_100 = []
for i in range(6,9):
    for data in data_100[i]:
        ids_100.append(data[0])

ids_100 = list(set(ids_100))
print(len(ids_100))

4930


In [ ]:
# get pickle file
with open('9_way_pdf_dict.pkl', 'rb') as f:
    dict_ids_9 = pickle.load(f)
    
url_dict_9={}
not_valid_ids_9 = []
for i in ids_9:
    try:
        url_dict_9[i]=dict_ids_9[i]
    except:
        not_valid_ids_9.append(i)
        continue
print(len(ids_9))
print(len(url_dict_9))
print(len(not_valid_ids_9))

In [4]:
# get pickle file
with open('100_way_pdf_dict.pkl', 'rb') as f:
    dict_ids_100 = pickle.load(f)
    
url_dict_100={}
not_valid_ids_100 = []
for i in ids_100:
    try:
        url_dict_100[i]=dict_ids_100[i]
    except:
        not_valid_ids_100.append(i)
        continue
print(len(ids_100))
print(len(url_dict_100))
print(len(not_valid_ids_100))

4930
4094
836


In [7]:
# Define the directory path
directory_path = "/mnt/data0/Datasets/imslp/score_scrape/results/composer"

# Change the current working directory
os.chdir(directory_path)

# Dictionary to store results
results = {}

for number in not_valid_ids_100:
    # Define the command with the current number
    command = f"find . -name '{number}.*'"
    
    # Execute the command and capture the output
    result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    
    # Check if the command was successful
    if result.returncode == 0:
        # Store the output in the dictionary
        results[number] = result.stdout.splitlines()
    else:
        # Print error message if command failed
        print(f"Error executing command for {number}: {result.stderr}")

{'314718': ['./Merkel,_Gustav_Adolf/Fantasie_No.5,_Op.176_/314718.pdf'], '95236': ['./Chavagnat,_Edouard/Aubade,_Op.176_/95236.pdf'], '32805': ['./Wolff,_%C3%89douard/Nocturne_et_romanesca,_Op.109_/32805.pdf'], '380849': ['./Maylath,_Henry/Petite_Histoirette,_Op.168_/380849.pdf'], '54638': ['./Smith,_Sydney/Grande_Fantaisie_on_%27Guillaume_Tell%27,_Op.61_/54638.pdf'], '13265': ['./Hummel,_Johann_Nepomuk/An_Alexis_/13265.pdf'], '251028': ['./Wallace,_William_Vincent/Fantasie_brillante_sur_%22La_traviata%22_/251028.pdf'], '08725': ['./Dubois,_Th%C3%A9odore/12_Pi%C3%A8ces_Nouvelles_pour_Orgue_/08725.pdf', './Gouin,_Pierre/12_Pi%C3%A8ces_Nouvelles_pour_Orgue_/08725.pdf'], '409526': ['./Lamothe,_Georges/Chant_des_Alpes_/409526.pdf'], '54650': ['./Smith,_Sydney/Rossini%27s_Stabat_Mater,_Paraphrase,_Op.89_/54650.pdf'], '367416': ['./Merkel,_Gustav_Adolf/Lenz_und_Lied,_Op.120_/367416.pdf'], '516214': ['./Battmann,_Jacques-Louis/Anna-polka_/516214.pdf'], '209523': ['./Oesten,_Theodore/Fleurs_de

In [23]:
# Construct urls out of File names
def create_url_from_file_name(path):
    components = path.strip('./').split('/')
    title = components[-2]
    composer = components[-3]
    url = f"https://imslp.org/wiki/{title}({composer})"
    return url

In [27]:
for key, file in results.items():
    try:
        url_dict_100[key] = create_url_from_file_name(file[0])
    except:
        print(key,file)

19111 []
82213 []


In [29]:
# functions to get page text and parse out the general information sectioin

def get_page_text(url):
    '''Retrive the text from url.'''
    # parse web content into English text
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for any request errors

        soup = BeautifulSoup(response.content, 'html.parser')
        # Extract all text from the page, excluding script and style tags
        text = soup.get_text(separator='\n', strip=True)

        return text
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

def extract_text_between_strings_second(text, start_string, end_string):
    '''
    Extracts the substring between the second occurrence of start_string and end_string in text.
    '''
    # Find the second occurrence of the start_string (the second occurence of "general information" gives us our metadata)
    if text:
        start_index = text.find(start_string, text.find(start_string) + len(start_string))
        if start_index == -1:  # If start_string is not found
            return None

        # Find the occurrence of the end_string
        end_index = text.find(end_string, start_index)
        if end_index == -1:  # If end_string is not found
            return None

        # Extract the substring between the second occurrence of start_string and end_string
        extracted_text = text[start_index + len(start_string):end_index].strip()

        return extracted_text

def retrieve_general_information(url):
    '''Retrieves the general information section from the IMSLP page of a musical score until Instrumentation.'''
    # retrieve the general information section
    parsed_text = extract_text_between_strings_second(get_page_text(url),'General Information','Instrumentation')
    return parsed_text

def extract_text_between_strings_new(text, start_string, end_string):
    '''
    Extracts the substring between start_string and end_string in text.
    '''
    start_index = text.find(start_string)
    if start_index == -1:  # If start_string is not found
        return None

    # Find the occurrence of the end_string
    end_index = text.find(end_string, start_index)
    if end_index == -1:  # If end_string is not found
        return None

    # Extract the substring between the second occurrence of start_string and end_string
    extracted_text = text[start_index + len(start_string):end_index].strip()

    return extracted_text

In [30]:
# find general information section

def get_gen_info(url_dict):
    attribute_list = ['Work Title', 'Alt\nernative\n.\nTitle', 'Name Translations', 'Name Aliases', 'Authorities', 'Composer',
                            'Opus/Catalogue Number', 'I-Catalogue Number', 'Key', 'Movements/Sections', 'Fisrt Performance', 'Year/Date of Composition', 
                            'First Pub\nlication', 'Librettist', 'Language', 'Copyright Information','Dedication', 'Average Duration', 'Composer Time Period', "Piece Style"]
    list_of_deletable_headings = ['Opus/Catalogue Number','I-Catalogue Number','Movements/Sections','Year/Date of Composition','First Pub\nlication','Average Duration','Composer Time Period']
    bad_heading_pattern = r'^.*\n'

    info = {}
    for pdf_id, url in url_dict.items():
        parsed_text = retrieve_general_information(url)

        if parsed_text:

            info_dic = {}

            present_attributes = []
            # check which attributes are present in the parsed text
            for attribute in attribute_list:
                if parsed_text.find(attribute) != -1:
                    present_attributes.append(attribute)

            # extract the text for each present attribute
            for attribute in attribute_list[:-1]:
                if attribute in present_attributes:
                    info_dic[attribute] = extract_text_between_strings_new(parsed_text, attribute, present_attributes[present_attributes.index(attribute) + 1] if present_attributes.index(attribute) != len(present_attributes) - 1 else "Romantic")

            info_dic['Piece Style'] = parsed_text[parsed_text.find('Piece Style')+12:] # extract the piece style from the end of the general information section
            info_dic['Instrumentation'] = 'Piano'
            info_dic['url'] = url

            if 'Alt\nernative\n.\nTitle' in present_attributes:
                alt_title = info_dic['Alt\nernative\n.\nTitle']
                info_dic['Alternative Title'] = alt_title
                del info_dic['Alt\nernative\n.\nTitle']

            if 'First Pub\nlication' in present_attributes:
                alt_title = info_dic['First Pub\nlication']
                info_dic['First Publication'] = alt_title
                del info_dic['First Pub\nlication']

            for heading in list_of_deletable_headings:
                if heading in info_dic.keys():
                    try:
                        info_dic[heading] = re.sub(bad_heading_pattern, '', info_dic[heading])
                    except:
                        pass

            info[pdf_id] = info_dic

        else:
            print("String not found or invalid input.")
            
    return info

In [ ]:
info_9 = get_gen_info(url_dict_9)

In [31]:
info_100 = get_gen_info(url_dict_100)

Error: 404 Client Error: Not Found for url: https://imslp.org/wiki/An_Alexis_(Hummel,_Johann_Nepomuk)
String not found or invalid input.
Error: 404 Client Error: Not Found for url: https://imslp.org/wiki/Trio_in_G_minor_(Gouin,_Pierre)
String not found or invalid input.
Error: 404 Client Error: Not Found for url: https://imslp.org/wiki/2_(Conservatoristen-Polka,_Op.1)
String not found or invalid input.
Error: 404 Client Error: Not Found for url: https://imslp.org/wiki/A_toi_(Smith,_Sydney)
String not found or invalid input.
Error: 404 Client Error: Not Found for url: https://imslp.org/wiki/Suite_Espa%C3%B1ola_No.1,_Op.47_(Gouin,_Pierre)
String not found or invalid input.
String not found or invalid input.
Error: 404 Client Error: Not Found for url: https://imslp.org/wiki/Suite_bergamasque_(Gouin,_Pierre)
String not found or invalid input.
Error: 404 Client Error: Not Found for url: https://imslp.org/wiki/Espagne_(Alb%C3%A9niz,_Isaac)
String not found or invalid input.
Error: 404 Client

In [33]:
# Manually fix error in works

fixed_url_100 = {}

# 2 pieces got deleted

# Pierre Gouin is not the composer but the arranger
def find_key_by_value(dictionary, target_value):
    for key, value in dictionary.items():
        if value == target_value:
            return key
    return None

fixed_url_100[find_key_by_value(url_dict_100, 'https://imslp.org/wiki/Trio_in_G_minor_(Gouin,_Pierre)')] = 'https://imslp.org/wiki/Trio_in_G_major%2C_BWV_1027a_(Bach%2C_Johann_Sebastian)'
fixed_url_100[find_key_by_value(url_dict_100,'https://imslp.org/wiki/Suite_Espa%C3%B1ola_No.1,_Op.47_(Gouin,_Pierre)')]= 'https://imslp.org/wiki/Suite_espa%C3%B1ola_No.1%2C_Op.47_(Alb%C3%A9niz%2C_Isaac)'
fixed_url_100[find_key_by_value(url_dict_100,'https://imslp.org/wiki/Suite_bergamasque_(Gouin,_Pierre)')]= 'https://imslp.org/wiki/Suite_bergamasque_(Debussy,_Claude)'


# parsing error
fixed_url_100[find_key_by_value(url_dict_100, 'https://imslp.org/wiki/2_(Conservatoristen-Polka,_Op.1)')] = 'https://imslp.org/wiki/Conservatoristen-Polka%2C_Op.1%2F2_(Moszkowski%2C_Moritz)'

# piece name changed to English
fixed_url_100[find_key_by_value(url_dict_100,'https://imslp.org/wiki/A_toi_(Smith,_Sydney)')]= 'https://imslp.org/wiki/For_You_(Smith%2C_Sydney)'
fixed_url_100[find_key_by_value(url_dict_100,'https://imslp.org/wiki/Espagne_(Alb%C3%A9niz,_Isaac)')]= 'https://imslp.org/wiki/Espa%C3%B1a%2C_Op.165_(Alb%C3%A9niz%2C_Isaac)'
fixed_url_100[find_key_by_value(url_dict_100,'https://imslp.org/wiki/1_(Aspiration,_Op.208)')]= 'https://imslp.org/wiki/Aspiration%2C_Op.208%2F1_(Smith%2C_Sydney)'

# changed url format
fixed_url_100[find_key_by_value(url_dict_100,'https://imslp.org/wiki/Raccolta_di_sonate_per_gli_organi_moderni_(Morandi,_Giovanni)')]= 'https://imslp.org/wiki/Raccolta_di_sonate_per_gli_organi_moderni_(1846)_(Morandi%2C_Giovanni)'

In [35]:
added_info_100 = get_gen_info(fixed_url_100)

In [37]:
info_100.update(added_info_100)
url_dict_100.update(fixed_url_100)
len(info_100)

4926

In [ ]:
# create pickle file from the dictionary
with open('9_way_metadata.pkl', 'wb') as f:
    pickle.dump(info_9, f)

In [44]:
with open('/home/ctang/ttmp/MMCSR/100_way_metadata.pkl', 'wb') as f:
    pickle.dump(info_100, f)

In [ ]:
# create a dataframe from the dictionary
df_9 = pd.DataFrame.from_dict(info_9, orient='index')
df_9.info()

# save the dataframe to a csv file
df_9.to_csv('9_way_metadata.csv')

In [46]:
# create a dataframe from the dictionary
df_100 = pd.DataFrame.from_dict(info_100, orient='index')
df_100.info()

# save the dataframe to a csv file
df_100.to_csv('/home/ctang/ttmp/MMCSR/100_way_metadata.csv')

<class 'pandas.core.frame.DataFrame'>
Index: 4926 entries, 06407 to 106714
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Work Title                4926 non-null   object
 1   Composer                  4926 non-null   object
 2   Opus/Catalogue Number     4316 non-null   object
 3   I-Catalogue Number        4923 non-null   object
 4   Key                       2478 non-null   object
 5   Movements/Sections        4076 non-null   object
 6   Composer Time Period      4926 non-null   object
 7   Piece Style               4926 non-null   object
 8   Instrumentation           4926 non-null   object
 9   url                       4926 non-null   object
 10  Alternative Title         4926 non-null   object
 11  First Publication         3711 non-null   object
 12  Name Translations         403 non-null    object
 13  Name Aliases              305 non-null    object
 14  Authorities            